In [ ]:
from datetime import datetime
import pytz

def calculate_time_difference(location_1, location_2):
    time1 = datetime.now(pytz.timezone(location_1))
    time2 = datetime.now(pytz.timezone(location_2))

    # Calculate the difference in hours and minutes
    time_difference_in_minutes = ((time2.hour * 60 + time2.minute) - (time1.hour * 60 + time1.minute))

    hours, minutes = divmod(abs(time_difference_in_minutes), 60)

    # Determine if the time is ahead or behind local time
    difference = "behind" if time_difference_in_minutes < 0 else "after"
    time_difference_str = f'{hours} hours, {minutes} minutes {difference}'

    return time_difference_str

In [ ]:
import openai, json

local_time = datetime.now()

def conversation_with_time():
    
    available_functions = { "calculate_time_difference": calculate_time_difference }
    functions = [
                    {
                        "name": "calculate_time_difference", # function name
                        "description": "Calculates the time difference for a given location", # one liner to describe function task
                        "parameters": {
                            "type": "object",
                            "properties": { # arguments
                                "location_1": { # argument name
                                    "type": "string", # argument datatype
                                    "description": "Timezone to be compared e.g. Asia/Tokyo" # argument description
                                },
                                "location_2": {
                                    "type": "string",
                                    "description": "Timezone to be compared e.g. Asia/Tokyo",
                                },
                            },
                            "required": ["location_1","location_2"], # update required parameters
                        },
                    }
                ]
    
    messages = [{"role": "user", "content": "What's the time difference between Singapore and Tokyo"}]

    # Get initial response
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613",messages=messages,functions=functions)
    response_message = response["choices"][0]["message"]
    print(response)
    print("="*50)

    if response_message.get("function_call"):
        # TODO: hanle JSON response errors
        
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call( # updated this method
                                            location_1=function_args.get("location_1"),
                                            location_2=function_args.get("location_2")
                                           )

        # Append Assistant reply and Function message to existing message
        messages.append(response_message)
        messages.append({"role": "function", "name": function_name, "content": function_response})
        print(response)
        print("="*50)

        # Get second response
        second_response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613", messages=messages)
        print(second_response)
        print("="*50)
        return second_response

print(conversation_with_time()["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-7kuIPx5W1xGT6e1GYOmlPEV0ajvD6",
  "object": "chat.completion",
  "created": 1691413845,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "calculate_time_difference",
          "arguments": "{\n  \"location_1\": \"Asia/Singapore\",\n  \"location_2\": \"Asia/Tokyo\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 94,
    "completion_tokens": 33,
    "total_tokens": 127
  }
}
{
  "id": "chatcmpl-7kuIPx5W1xGT6e1GYOmlPEV0ajvD6",
  "object": "chat.completion",
  "created": 1691413845,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "calculate_time_difference",
          "arguments": "{\n  \"location_1\": \"Asia/Singapore\",\n  \"locatio